# Language Agent Tree Search

This notebook is an example implementation of a [LangChain](https://github.com/hwchase17/langchain) [Lateral Agents Tree Search](https://github.com/langchain-ai/langgraph/blob/main/examples/lats/lats.ipynb) powered by [Anthropic's Claude](https://www.anthropic.com/news/claude-3-family).

[Language Agent Tree Search](https://arxiv.org/abs/2310.04406) (LATS), by Zhou, et. al, is a general LLM agent search algorithm that combines reflection/evaluation and search (specifically monte-carlo trees search) to get achieve better overall task performance compared to similar techniques like ReACT, Reflexion, or Tree of Thoughts.

## Prerequisites

Install `langgraph` (for the framework), `langchain_anthropic` (for the LLM), and `langchain` + `tavily-python` + `kay` + `yfinance`.

We will use tavily search as a tool. You can get an API key [here](https://app.tavily.com/sign-in) or replace with a different tool of your choosing.

[Kay](https://kay.ai/) is used to search SEC Filings and Press Releases of US companies. You will need an API key: you can get one for free at [https://kay.ai](https://kay.ai/). Once you have an API key, you must set it as an environment variable `KAY_API_KEY`.

[Yahoo finance news](https://finance.yahoo.com/) is used to find financial news about a public company.

In [1]:
# %pip install -U --quiet  langchain langgraph langchain_openai
# %pip install -U --quiet tavily-python kay yfinance

In [2]:
import sys
import logging

stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(stream_handler)

In [4]:
import getpass
import os


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)

_set_if_undefined("ANTHROPIC_API_KEY")
_set_if_undefined("TAVILY_API_KEY")
_set_if_undefined("KAY_API_KEY")

## How it works

### Classes and Methods

#### CandidatesGenerator

The [CandidatesGenerator](../../../../../slangchain/graphs/anthropic/lats/lats.py) class is responsible for generating candidate responses given a prompt. It interfaces with a language model (LLM) and generates multiple candidate responses asynchronously.

- `generate_candidates`: Generates candidate responses synchronously.
- `from_tools`: Constructs a CandidatesGenerator instance from a sequence of BaseTool objects.

#### Node

The Node class represents a node in the search tree. It stores information about messages exchanged in the conversation, reflections on responses, and handles tree traversal and backpropagation.

#### LATS

The [LATS](../../../../../slangchain/graphs/anthropic/lats/lats.py)  class is the main orchestrator of the tree search process. It initializes the workflow, defines the expansion and termination conditions, and executes the search.

- `_call`: Executes the LATS search process.
- `from_tools`: Constructs an LATS instance from a sequence of BaseTool objects.

### Workflow

The [LATS](../../../../../slangchain/graphs/anthropic/lats/lats.py) class orchestrates a tree-based search mechanism designed to generate and refine AI-generated responses. Here’s a breakdown of its methods and operational flow:

#### Methods

1. **`generate_initial_response(self, state: TreeState) -> Dict`**
   - **Purpose**: Generates the initial response for the user's input and initializes the root node of the search tree.
   - **Process**:
     - Invokes the `initial_answer_chain` to get a preliminary response based on user input.
     - Parses the response and manages any necessary tool invocations.
     - Creates a `Node` representing the root of the search tree using the response and reflection data.

<br>

2. **`expand(self, state: TreeState, config: RunnableConfig) -> TreeState`**
   - **Purpose**: Expands the existing search tree by generating new candidate responses from the current best node and exploring them.
   - **Process**:
     - Identifies the best candidate node to expand from.
     - Generates new candidates via the `generate_candidates_chain` based on the messages of the best node.
     - Parses and executes tool responses for each new candidate.
     - Creates new nodes for each candidate and updates the tree structure.

<br>

3. **`_should_loop(self, state: TreeState)`**
   - **Purpose**: Determines whether the search should continue.
   - **Criteria**:
     - Checks if the root node has solved the problem.
     - Checks if the tree has reached its maximum configured height.

<br>

4. **`init_workflow_nodes(self) -> StateGraph`**
   - **Purpose**: Initializes the nodes and edges in the workflow graph, which dictates the sequence of operations in the tree search.
   - **Configuration**:
     - Adds nodes like 'start' and 'expand' with corresponding methods.
     - Configures conditional edges to decide whether to continue expanding or complete the search.

<br>

5. **`_call(self, inputs: Dict[str, Any], run_manager: Optional[CallbackManagerForChainRun] = None) -> Dict[str, Any]`**
   - **Purpose**: Executes the tree search based on the given inputs.
   - **Process**:
     - Initializes and compiles the workflow graph.
     - Processes each step in the graph, dynamically moving through 'start', 'expand', or ending the process based on tree state.

#### Flow of Operation

1. **Initialization**
   - An instance of LATS is configured with specific tools, model settings, and parameters like `tree_height` and `reflection_retries`.

<br>

2. **Starting the Search**
   - The process begins at the `start` node, where `generate_initial_response` is called to create the root of the tree based on the initial user query.

<br>

3. **Tree Expansion**
   - The tree is expanded by iterating through the `expand` node. At each step:
     - The best node to expand is selected based on specific criteria (e.g., highest value, least visited).
     - New candidate nodes are generated and added as children of the best node.
     - Reflections are performed on new nodes to evaluate and score responses.

<br>

4. **Looping Decision**
   - After each expansion, `_should_loop` is evaluated to decide whether to continue expanding or conclude the search. The search might end if the problem is solved (a node meets the criteria) or if the maximum tree depth is reached.

<br>

5. **Conclusion**
   - The search concludes when `_should_loop` determines no further expansion is needed. The best solution or the state of the search is then returned as the final output.

By structuring the search this way, LATS efficiently manages a complex decision-making process, allowing for iterative refinement of AI-generated responses in a controlled and scalable manner.

#### Tools

In the below example, the LATS (Language-Agnostic Tool Set) is being set up to perform a specific task: generating a report on Boeing stock based on a given query. Let's break down the tools provided to LATS and their objectives:

1. We will use tavily search as a tool. You can get an API key [here](https://app.tavily.com/sign-in) or replace with a different tool of your choosing.

2. [Kay](https://kay.ai/) is used to search SEC Filings and Press Releases of US companies. You will need an API key: you can get one for free at [https://kay.ai](https://kay.ai/). Once you have an API key, you must set it as an environment variable `KAY_API_KEY`.

3. [Yahoo finance news](https://finance.yahoo.com/) is used to find financial news about a public company.


In [5]:
"""URL Compressed Search tool"""
import json
from typing import Optional, Dict, List, Type
from pydantic import Field, BaseModel

from langchain.callbacks.base import BaseCallbackHandler
from langchain.callbacks.manager import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun
from langchain.tools.base import BaseTool

from langchain_community.retrievers import KayAiRetriever

class KayDocSearchInput(BaseModel):
  """Input for WikipediaDocSearch."""
  query: str = Field(
    ...,
    description=(
      "Input is a search query,"
    ))

class KayDocSearch(BaseTool):
  """Tool that has capability to request web content 
  and return the text that is most similar to the query."""

  args_schema: Type[BaseModel] = KayDocSearchInput

  k: int = Field(default = 5)

  input_key: str = Field(default = "query")
  output_key: str = Field(default = "result")

  metadata: Optional[Dict] = Field(default = None)

  name : str = "SEC_Filings_Search"
  description : str = """
    A wrapper around SEC filings and press release search.
    Useful for when you need to answer questions about
    SEC filings, US companies' financial info, and press releases of US companies."""

  @classmethod
  def from_parameters(
    cls,
    k: int = 3,
    metadata: Optional[Dict] = None,
    callbacks: List[BaseCallbackHandler] = None):
    """from parameters"""
    return cls(
      k = k,
      metadata = metadata,
      callbacks = callbacks)

  def _run(
    self,
    query: str,
    run_manager: Optional[CallbackManagerForToolRun] = None,) -> str:
    """Use the tool."""

    retriever = KayAiRetriever.create(
      dataset_id="company",
      data_types=["10-K", "10-Q", "PressRelease"],
      num_contexts=self.k)

    result = retriever.get_relevant_documents(query)
    return json.dumps([ doc.dict() for doc in result ])

  async def _arun(
    self,
    query: str,
    run_manager: Optional[AsyncCallbackManagerForToolRun] = None,) -> str:
    """Use the tool asynchronously."""
    raise NotImplementedError("KayDocSearch does not support async")

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
from langchain.callbacks.stdout import StdOutCallbackHandler


search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=2)

class YahooFinanceNewsInput(BaseModel):
  """Input for YahooFinanceNewsTool."""
  query: str = Field(
    ...,
    description=(
      "Input is a search query or ticker symbol"
    ))


yahoo_finance_tool = YahooFinanceNewsTool()
yahoo_finance_tool.args_schema = YahooFinanceNewsInput

sec_search = KayDocSearch.from_parameters(k = 2)

tools = [tavily_tool, yahoo_finance_tool, sec_search]

## Invoke

These tools are then passed to the LATS instance to be utilised collectively in generating a report. The LATS instance is configured with parameters like the model name, tree height, and settings related to candidate completions. It's meant to facilitate the integration of multiple tools for a unified task. Once set up, the LATS instance is invoked with a specific input query, triggering the execution of the configured tools to generate a report on Boeing stock.

In [7]:
import nest_asyncio

nest_asyncio.apply()

from slangchain.graphs.anthropic.lats.lats import LATS

lats = LATS.from_tools(
  tools = tools,
  tree_height = 3,
  candidate_completions_num = 3,
  candidate_completions_temperature = 0.6
)

result = lats.invoke(input={"input": "Write a report on Boeing stock. Analyse the price and company's revenue to report on if I should buy the stock."})

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


NumExpr defaulting to 8 threads.
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BOEING%20STOCK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BOEING+STOCK&crumb=QSf0XZOs.po

start

rolled out: 1
---


expand

rolled out: 2
---


expand

rolled out: 3
---


expand

rolled out: 3
---


expand

rolled out: 4
---



In [8]:
step = result["output"]
step["expand"]["root"].get_best_solution()

<Node value=0.9, visits=1, solution=[AIMessage(content="Based on the additional research, here is a more comprehensive analysis on whether you should buy Boeing stock:\n\nBoeing's Financial Performance:\n- Boeing has faced significant challenges in recent years, including the 737 MAX grounding and the COVID-19 pandemic, which have impacted their financial results. \n- In Q1 2023, Boeing reported a net loss of $425 million, though this was an improvement from the $1.2 billion loss in Q1 2022.\n- Boeing's revenue and profit margins have been depressed, but they are showing signs of gradual recovery as air travel demand rebounds.\n\nStock Price and Valuation:\n- Boeing's stock price has been volatile, trading in the $150-$250 range over the past year.\n- The stock is currently trading at a price-to-earnings (P/E) ratio of around 50, which is high compared to the overall market and Boeing's historical averages.\n- Analysts have a median price target of around $220, suggesting the stock may

In [9]:
print(f'{step["expand"]["root"].get_best_solution().messages[0].content}')

Based on the additional research, here is a more comprehensive analysis on whether you should buy Boeing stock:

Boeing's Financial Performance:
- Boeing has faced significant challenges in recent years, including the 737 MAX grounding and the COVID-19 pandemic, which have impacted their financial results. 
- In Q1 2023, Boeing reported a net loss of $425 million, though this was an improvement from the $1.2 billion loss in Q1 2022.
- Boeing's revenue and profit margins have been depressed, but they are showing signs of gradual recovery as air travel demand rebounds.

Stock Price and Valuation:
- Boeing's stock price has been volatile, trading in the $150-$250 range over the past year.
- The stock is currently trading at a price-to-earnings (P/E) ratio of around 50, which is high compared to the overall market and Boeing's historical averages.
- Analysts have a median price target of around $220, suggesting the stock may be slightly overvalued at current levels.

Risks and Opportunitie